### FE621 - Homework #3

**Author**: Sid Bhatia

**Date**: April 5th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Sveinn Olafsson

**TA**: Dong Woo Kim

#### Problem #1 (Monte Carlo Error)

Use Monte Carlo simulation to price a European call option in the Black-Scholes model with the following parameters: $S_0 = 100, \sigma = 0.30, r = 0.05, T = 1,$ and $K = 100$.

a. Use (exact) simulation based on the closed-form solution of geometric Brownian motion. Use $n = 100,000$ paths. 

Clearly describe the steps of your simulation procedure, and provide formulas for the Monte Carlo estimator and a corresponding 95% confidence interval. Report both the estimator and the confidence interval. Does the confidence interval contain the true price of the option?

##### Procedure

1. **Simulation of Stock Prices**: According to BSM, the stock process $S_t$ at future time $t$ is as follows:

$$S_t = S_0 \exp\{(r - \frac{1}{2}\sigma^2)T + \sigma \sqrt{T} W_T\}$$

where

- $S_0$ = initial stock price
- $r$ = rfr (e.g., 3-month UST)
- $\sigma$ = vol
- $T$ = time till maturity
- $W_T$ = standard BM $\sim \mathcal{N}(0, T)$

2. **Payoff Calculation**: For a call option, the payoff at maturity is $(S_T - K)_+$ where $K$ is the strike price. For puts, it's the converse $(K - S_T)_+$.

3. **MC Estimator**: The price of the option is the present value of the expected payoff under the risk-neutral measure $\mathbb{Q}$, which is estimated as the average of the discounted payoffs across all simulated paths:

$$P = e^{-rT} \mathbb{E}^Q[f(S_t)], \; \text{where} \; f \; \text{is the payoff function}. \\ \dots \\ \hat{C} = \exp\{(-rT)\} \frac{1}{n} \sum_{i = 1}^n f(S_t)$$

4. **CI**: The 95% confidence interval for the true option price is given by $$\hat{C} \pm z_{\alpha / 2} \cdot SE$$ 

where $\alpha = 0.05$ and $SE = \text{standard error}$. Therefore,

$$\hat{C} = 1.96 \cdot \frac{\sigma_{\hat{C}}}{\sqrt{n}}$$

where $\sigma_{\hat{C}}$ is the standard deviation of the stimulated payoffs.